# Install Tensorflow 2.0 with GPU support to RUN MaskRCNN
### Created by Quanyuan He (2021/9/1)
### Accessed from https://github.com/hqyone/Mask_RCNN_TF2/

# 0. Get System infomation
```
    # Show all drivers
    !ubuntu-drivers devices
    # get cuda version
    !nvcc --version
    # check ubuntu version
    !lsb_release -a
```

# 1. Check the version combination
### Tested build configurations can be found here: https://www.tensorflow.org/install/source#gpu
### Please make sure all version is matched!!!!

# 2. Setup python enviroment
* Conda 4.10.1
* Tersorflow=2.6 (current last version)
* scikit-image = 0.18  (may be 0.16.2 is better to avoid warning message)


## Download anaconda install bash file from https://www.anaconda.com/products/individual#Downloads
```
        sudo chmod +x Anaconda3-2021.05-Linux-x86_64.sh
        ./Anaconda3-2021.05-Linux-x86_64.sh
```
## create a environment using python 3.8 
```
        conda create -n tf2 python=3.8
```
## Install other required package, here tensorflow will be 2.6
```
        python -m pip install tensorflow
        python -m pip install -U scikit-image
        python -m pip install imagaug
        python -m pip install pycocotools
        python -m pip install jupyter
        pip install git
```

# 3. Install CUDA and cuDNN8.1
1. follow the url (https://www.tensorflow.org/install/gpu) to insall all driver and packages for Ubuntu 18.04 (CUDA 11.0), it is ok for ubuntu 20
*  you can check the conditions by
````
        nvidia-smi  
````
2. The step 1 only installs cuDNN 8.0, but we need cuDNN 8.1
````
        Download deb file from evidia website (libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb) to re-install these two package.
        sudo apt install /home/hqyone/mnt/2tb/hunnu/lxg/tools/nvidia/libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb
````
3. make sure the cuda path and lib path directed to correct folder
```
        vim ~/.bashrc
        export PATH=/usr/local/cuda-11.0/bin:$PATH
        export LD_LIBRARY_PATH=/usr/local/cuda-11.0/targets/x86_64-linux/lib
```
4. Fix can't find libcusolver.so.11 issue
```
        !sudo ln -s /usr/local/cuda-11.0/lib64/libcusolver.so.10 /usr/local/cuda-11.0/lib64/libcusolver.so.11
```
5. reboot
6. Check the available of GPU for tensorflow
```
        tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)
        tf.config.list_physical_devices('GPU')
```

# 4 Download and modified Maskrcnn repository
* Serveral version of MaskRCNN are acknowledged can be used by Tersorflow 2.0
* However, there are some bugs need be fixex before wrong it
* Download the git clone code or using this version directly
```
    git clone the https://github.com/leekunhee/Mask_RCNN
```
* Fix the memory leaking issue.
    Add following code to the begining of model.py
    ```
    import tensorflow as tf
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if len(gpus)>0:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    ```
* Fix the out of index issue in generate mask for balloon project
    Open balloon.py insert following code below the line 167
    ```
    rr = np.clip(rr, 0, info["height"]-1)
    cc = np.clip(cc, 0, info["width"]-1)
    ```
* model.py line 2378-2379 need be changed to:
```
    workers=1,
    use_multiprocessing=False,
```


# 5. get rid of any NVIDIA/CUDA packages you may already have installed
### Uninstall CUDA
```
    !sudo rm /etc/apt/sources.list.d/cuda*
    !sudo apt remove --autoremove nvidia-cuda-toolkit
    !sudo apt remove --autoremove nvidia-*
```
### Purge any remaining NVIDIA configuration files and the associated dependencies that they may have been installed with.
```
    sudo apt-get purge nvidia*
    sudo apt-get autoremove
    sudo apt-get autoclean
```
### Remove any existing CUDA folders you may have in /usr/local/
```
    sudo rm -rf /usr/local/cuda*
```